In [1]:
from selenium import webdriver
from utils import (
    lighthouse_utils,
    webscraping_utils,
    annotations_utils,
    web_utils,
    selenium_utils,
    const,
)

In [2]:
from selenium.webdriver.remote.webelement import WebElement

In [3]:
url = "https://vercel.com"

In [6]:
driver = selenium_utils.initialize_driver()

annotation = annotations_utils.generate_annotation_from_url(driver, url)


driver initialized


In [6]:
bounding_boxes_dict = webscraping_utils.generate_bounding_boxes_of_tags(
    driver, const.TARGET_TAGS
)
lighthouse_score = lighthouse_utils.get_lighhouse_score()
annotation = annotations_utils.generate_annotation(
    url, lighthouse_score, bounding_boxes_dict
)

Invalid h1 size found.


In [11]:
annotations_utils.make_annotation_on_csv_file(annotation)

In [ ]:
driver.quit()